# Scrape the current data


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [109]:

url = "https://news.ycombinator.com/item?id=36573871"

dataLocation = "currentData/"
month = "July"
postsTableName = "posts_" + month

In [110]:
import psycopg2

conn = psycopg2.connect("dbname=Bumpant user=Bumpant password=ampegskb")
cursor = conn.cursor()

query = """CREATE TABLE IF NOT EXISTS """ + postsTableName + """ (
        id SERIAL PRIMARY KEY, 
        hnuser TEXT, 
        date TIMESTAMP, 
        comment TEXT,
        fileName TEXT,
        html TEXT
        );"""

cursor.execute(query)

conn.commit()   
conn.close() 

In [113]:
from hn_jobs_chat.scraper import scrape

# April 2024
monthPosts = scrape(url)
monthPosts = monthPosts[0:154]

Scraping page 1
Scraping page 2
all pages scraped
Scraped 155 posts


In [114]:
import psycopg2

conn = psycopg2.connect("dbname=Bumpant user=Bumpant password=ampegskb")
cursor = conn.cursor()

for post in monthPosts:
    hnuser = post["hnuser"]
    date = post["date"]
    comment = post["comment"]
    fileName = post["fileName"]
    html = post["html"]

    cursor.execute(
        """INSERT INTO """ + postsTableName + 
        """ (hnuser, date, comment, fileName, html) VALUES (%s, %s, %s, %s, %s)"""
        , (hnuser, date, comment, fileName, html)
    )

    conn.commit()

conn.close()


In [ ]:
#| hide
from nbdev import nbdev_export
nbdev_export()